In [1]:
# %load_ext autoreload
# # Enable autoreload for all modules
# %autoreload 2

In [2]:
from iris_environments.environments import get_environment_builder
import numpy as np
import ipywidgets as widgets
from functools import partial
from pydrake.all import (RigidTransform, Rgba, Sphere, RotationMatrix)

In [3]:
from functools import partial
from pydrake.all import (StartMeshcat,
                         RobotDiagramBuilder,
                         MeshcatVisualizer,
                         LoadModelDirectives,
                         ProcessModelDirectives,
                         RigidTransform,
                         RotationMatrix,
                         MeshcatVisualizerParams,
                         Role,
                         RollPitchYaw,
                         Meldis,
                         AddDefaultVisualization,
                         Box
                         )
import numpy as np
import os

def plant_builder_7dof_bins(usemeshcat = False):
    if usemeshcat:
        #meshcat = StartMeshcat()
        meldis = Meldis()
        meshcat = meldis.meshcat
    builder = RobotDiagramBuilder()
    plant = builder.plant()
    scene_graph = builder.scene_graph()
    parser = builder.parser()
    #parser.package_map().Add("cvisirisexamples", missing directory)
    file_p = "/home/peter/gitcspace/iris_benchmarks/iris_environments"
    directives_file = file_p+"/directives/7dof_bins_example_urdf.yaml"#FindResourceOrThrow() 
    # directives_file = file_p+"/directives/7dof_bins_example.yaml"#FindResourceOrThrow() 
    path_repo = file_p #os.path.dirname(os.path.abspath(__file__))
    parser.package_map().Add("iris_environments", path_repo+"/assets")
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)
    plant.Finalize()
    if usemeshcat:
        #par = MeshcatVisualizerParams()
        #par.role = Role.kIllustration
        #visualizer = MeshcatVisualizer.AddToBuilder(builder.builder(), scene_graph, meshcat, par)
        visualizer = AddDefaultVisualization(builder.builder(), meshcat)
    diagram = builder.Build()
    diagram_context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(diagram_context)
    diagram.ForcedPublish(diagram_context)
    return plant, scene_graph, diagram, diagram_context, plant_context, meshcat if usemeshcat else None



In [4]:
import sys
path_repo = "/home/peter/gitcspace/iris_benchmarks/iris_environments"
sys.path.append('/home/peter/gitcspace/cuciv0/bazel-bin/cuci/src/pybind/pycuci')
import pycuci_bindings as pycuci
cuci_parser = pycuci.URDFParser()
cuci_parser.register_package("iris_environments",  path_repo + "/assets/")
cuci_parser.parse_directives(path_repo+"/directives/7dof_bins_example_urdf.yaml")
pl = cuci_parser.build_plant()
kt = pl.getKinematicTree()
links = kt.get_links()
for l in links:
    print(l.name)

Successfully registered package 'iris_environments' with path: /home/peter/gitcspace/iris_benchmarks/iris_environments/assets/
world
iiwa::iiwa_link_0
iiwa::iiwa_link_1
iiwa::iiwa_link_2
iiwa::iiwa_link_3
iiwa::iiwa_link_4
iiwa::iiwa_link_5
iiwa::iiwa_link_6
iiwa::iiwa_link_7
wsg::body
wsg::left_finger
wsg::right_finger
iiwa::wsg_attach
shelves::shelves_body
shelves::top_and_bottom
shelf_origin
binR::bin_base
bin_originR
binL::bin_base
bin_originL
table::table_body
table_origin


In [5]:
from iris_environments.environments import env_names, get_robot_instance_names
from pydrake.all import SceneGraphCollisionChecker

currname = env_names[6]
plant_builder = get_environment_builder(currname)
plant, scene_graph, diagram, diagram_context, plant_context, meshcat = plant_builder_7dof_bins(usemeshcat=True)

scene_graph_context = scene_graph.GetMyMutableContextFromRoot(
    diagram_context)
robot_instances = get_robot_instance_names(currname)
checker = SceneGraphCollisionChecker(model = diagram, 
                                     robot_model_instances = [plant.GetModelInstanceByName(r) for r in robot_instances], 
                                     edge_step_size = 0.1)

INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Allocating contexts to support implicit context parallelism 20


# Run this cell and then use the sliders and the button to save the seed points

In [6]:


q = np.zeros(plant.num_positions()) 
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]*0.99
    q_high = plant.GetPositionUpperLimits()[i]*0.99
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, step=0.001, description=f"q{i}"))

col_col =  Rgba(0.8, 0.0, 0, 0.5)    
col_free =  Rgba(0.0, 0.8, 0.5, 0.5) 
def showres(qvis):
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    query = plant.get_geometry_query_input_port().Eval(plant_context)
    col = query.HasCollisions()
    if col:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_col)
    else:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_free)
    meshcat.SetTransform(f"/drake/visualizer/shunk",
                                   RigidTransform(RotationMatrix(),
                                                  np.array([0,0,2])))
    return col

def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=2.9373894, min=-2.9373894, step=0.001)

FloatSlider(value=0.0, description='q1', max=2.0734559999999997, min=-2.0734559999999997, step=0.001)

FloatSlider(value=0.0, description='q2', max=2.9373894, min=-2.9373894, step=0.001)

FloatSlider(value=0.0, description='q3', max=2.0734559999999997, min=-2.0734559999999997, step=0.001)

FloatSlider(value=0.0, description='q4', max=2.9373894, min=-2.9373894, step=0.001)

FloatSlider(value=0.0, description='q5', max=2.0734559999999997, min=-2.0734559999999997, step=0.001)

FloatSlider(value=0.0, description='q6', max=3.0237867, min=-3.0237867, step=0.001)

In [10]:
from pydrake.all import HPolyhedron
domain = HPolyhedron.MakeBox(plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits())
cuci_domain = pycuci.HPolyhedron(domain.A(), domain.b())

In [16]:

import time
import pickle
t1 = time.time()
nodes = pycuci.UniformSampleInHPolyhedraCuda([cuci_domain], cuci_domain.ChebyshevCenter(), 100000, 100)[0]
is_col_free = pycuci.CheckCollisionFreeCuda(nodes, pl.getMinimalPlant())

N = 100
nodes = nodes[:, np.where(is_col_free)[0]]
nodes = nodes[:, :N]
print(f"found {nodes.shape[1]} collision-free nodes")
vg_cuci = pycuci.VisibilityGraph(nodes, pl.getMinimalPlant(), 0.1, 5000000)

with open(f'vg_7_dof_bins_uniform_{N}_upper_triangle_only.pkl', 'wb') as f:
    data = {'adjacency': vg_cuci, 'nodes': nodes}
    pickle.dump(data,f)
# t2 = time.time()
# print('pycuci done')
# vg_drake = VisibilityGraph(checker, nodes, parallelize=True)
# t3 = time.time()

# print(f"time cuci: {t2-t1:.3f} \ntime drake: {t3 - t2: .3f}")
# for i in range(0, vg_cuci.shape[1]-1):
#     for j in range(i+1, vg_cuci.shape[1]):
#         assert(vg_cuci[i,j] == vg_drake[i,j])

execution time cuda no copy: 4 ms
execution time cuda no copy: 3 ms
found 100 collision-free nodes
num batches 1 
 num_edges_per_batch 4950 
, num_edges_to_check 4950 
time to insert results in matrix: 0 ms


In [17]:
import pickle
with open('tmp/7dof_bin_tsseeding_4000_0.05_1337.pkl', 'rb') as f:
    data = pickle.load(f)


In [18]:
q_tot = np.concatenate(tuple(data['q_obj']), axis = 0)

In [31]:
import time
import pickle
t1 = time.time()
# nodes = pycuci.UniformSampleInHPolyhedraCuda([cuci_domain], cuci_domain.ChebyshevCenter(), 100000, 100)[0]
# is_col_free = pycuci.CheckCollisionFreeCuda(nodes, pl.getMinimalPlant())

N = 4000
# pts_binL = q_tot[:int(N/3), :]
# pts_binR = q_tot[4000:4000+int(N/3), :]
# pts_shelf = q_tot[8000:8000+N-2*int(N/3), :]
ts_samples = q_tot[8000:8000+N, :]
# ts_samples = np.concatenate((pts_binL, pts_binR, pts_shelf), axis = 0)

vg_cuci = pycuci.VisibilityGraph(ts_samples.T, pl.getMinimalPlant(), 0.1, 5000000)

with open(f'tmp/vg_7_dof_bins_shelves_only_{N}_upper_triangle_only.pkl', 'wb') as f:
    data = {'adjacency': vg_cuci, 'nodes': ts_samples.T}
    pickle.dump(data,f)

num batches 177 
 num_edges_per_batch 45046 
, num_edges_to_check 7998000 
time to insert results in matrix: 187 ms


(100, 7)

In [22]:
# showres(q_tot[4005]*dir)
import time
for q in ts_samples:
    showres(q)
    time.sleep(0.1)
    

In [11]:
is_col_free = pycuci.CheckCollisionFreeCuda(q_tot.T, pl.getMinimalPlant())
np.all(is_col_free)

execution time cuda no copy: 12 ms


True

In [32]:
counter = 0
for i in range(0, vg_cuci.shape[1]-1):
    for j in range(i+1, vg_cuci.shape[1]):
        if vg_cuci[i,j] != vg_drake[i,j]:
            counter +=1

In [33]:
counter

1

In [34]:
vg_cuci.shape[1]*(vg_cuci.shape[1]-1)/2

2595781.0

In [15]:
is_col_free

[0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,


In [13]:
pycuci.CheckCollisionFree(q*0, pl.getMinimalPlant())

True

In [6]:
q = np.array([-8.30000e-03,  1.16796e+00,  9.16740e-01,  8.10270e-01,
        8.84370e-01,  1.15137e+00,  6.28890e-01,  6.30620e-01,
       -3.00000e-04,  3.74960e-01,  1.42774e+00,  5.89270e-01,
       -1.43300e-01,  5.32960e-01,  9.91740e-01,  6.33270e-01])

In [7]:
file.close()

# Inspect seed points

In [8]:
import yaml
import time

with open(seed_point_file, 'r') as f:
    seed_points = yaml.safe_load(f)
seed_points = np.array(seed_points['seedpoints'])

for i, s in enumerate(seed_points):
    showres(s)
    print(f" point {i+1} / {len(seed_points)}")
    time.sleep(2)


 point 1 / 10
 point 2 / 10
 point 3 / 10
 point 4 / 10
 point 5 / 10
 point 6 / 10
 point 7 / 10
 point 8 / 10
 point 9 / 10
 point 10 / 10


In [11]:
plant.GetBodyByName("")

RuntimeError: GetRigidBodyByName(): There is no RigidBody named '' anywhere in the model (valid names in model instance 'WorldModelInstance' are: world; valid names in model instance 'allegro_hand' are: hand_root, link_0, link_1, link_10, link_11, link_12, link_13, link_14, link_15, link_2, link_3, link_4, link_5, link_6, link_7, link_8, link_9; valid names in model instance 'block' are: block_body)

In [20]:
showres(seed_points[-3])

False

In [10]:
q

array([ 0.434544, -0.6944  ])